In [34]:
import copy
from pprint import pprint

import numpy as np
import yaml
from easydict import EasyDict
from tqdm import tqdm

import wandb

In [35]:
api = wandb.Api()

In [36]:
with open("../configs/base_config.yaml", "r") as file:
    base_config = EasyDict(yaml.safe_load(file))

with open("../configs/igs_config.yaml", "r") as file:
    igs_config = EasyDict(yaml.safe_load(file))

with open("../configs/relative_flatness_config.yaml", "r") as file:
    relative_flatness_config = EasyDict(yaml.safe_load(file))

In [37]:
models = ["best_model", "model", "temp_best_model", "temp_model"]

In [38]:
metrics = ["acc", "ece", "squared_euclidean_norm", "relative_flatness", "IGS"]

In [39]:
approaches = ["no-regularisation", "augmentation", "dropout_0.1", "dropout_0.5", "weight_decay0.05", "weight_decay0.1"]

In [40]:
runs = api.runs(f"{base_config.wandb.entity}/VGG19_CIFAR10_")

In [41]:
results = {model: {approach: {metric: {} for metric in metrics} for approach in approaches} for model in models}

for run in tqdm(runs):
    for approach in approaches:
        if run.name.startswith(f"VGG19-CIFAR10-baseline-with-{approach}"):
            seed = run.name.split("-seed-")[-1]
            for metric in metrics:
                if metric in run.config['metrics']:
                    for model in models:
                        results[model][approach][metric][seed] = run.summary[f"eval_metrics_{model}/{metric}"]

100%|██████████| 42/42 [00:00<00:00, 12921.64it/s]


In [43]:
processed_results = copy.deepcopy(results)

for model in models:
    for approach in approaches:
        for metric in metrics:
            num_results = len(results[model][approach][metric])
            if num_results == 1:
                processed_results[model][approach][metric] = (list(results[model][approach][metric].values())[0], 0)
            else:
                processed_results[model][approach][metric] = (np.mean(list(results[model][approach][metric].values())),
                                                    np.std(list(results[model][approach][metric].values())))

In [67]:
prefix = r"""\begin{table}
    \centering
    \begin{tabular}{l>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}}
    \toprule
    \textbf{Regularizer} & \textbf{Accuracy} & \textbf{ECE} & \textbf{Squared euclidean norm} & \textbf{Relative flatness} & \textbf{IGS} \\ \midrule"""

for model in models:

    table = prefix

    for approach in approaches:
        if approach.startswith("dropout"):
            if approach.endswith("0.1"):
                table += f"\n\t\\textit{{Dropout}} \\\\"
            approach_name = "\\quad = " + approach.replace("dropout_", "")
        elif approach.startswith("weight_decay"):
            if approach.endswith("0.05"):
                table += f"\n\tWeight decay \\\\"
            approach_name = "\\quad = " + approach.replace("weight_decay", "")
        else:
            approach_name = approach
        approach_name = approach_name.replace("_", " ").replace("-", " ")
        approach_name_italicized_latex = f"\\textit{{{approach_name}}}"
        table += f"\n\t{approach_name_italicized_latex} "
        for metric in metrics:
            table += f"& {processed_results[model][approach][metric][0]:.2f}$_{{\pm {processed_results[model][approach][metric][1]:.2f}}}$ "
        table += r"\\"

    model = model.replace("_", " ")

    table += r"""
    \bottomrule \\
    \end{tabular}
    \caption{Results for VGG-19 on CIFAR-10 for """ + model + r"""}
    \label{tab:measures}
\end{table}
    """

    print(table)
    print("\n\n\n")

\begin{table}
    \centering
    \begin{tabular}{l>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}}
    \toprule
    \textbf{Regularizer} & \textbf{Accuracy} & \textbf{ECE} & \textbf{Squared euclidean norm} & \textbf{Relative flatness} & \textbf{IGS} \\ \midrule
	\textit{no regularisation} & 0.75$_{\pm 0.00}$ & 0.17$_{\pm 0.00}$ & 6.66$_{\pm 0.05}$ & 0.16$_{\pm 0.00}$ & 0.01$_{\pm 0.00}$ \\
	\textit{augmentation} & 0.73$_{\pm 0.01}$ & 0.10$_{\pm 0.01}$ & 5.93$_{\pm 0.05}$ & 14.56$_{\pm 0.74}$ & 13.52$_{\pm 0.00}$ \\
	\textit{Dropout} \\
	\textit{\quad = 0.1} & 0.76$_{\pm 0.01}$ & 0.17$_{\pm 0.01}$ & 7.19$_{\pm 0.08}$ & 0.16$_{\pm 0.01}$ & 0.01$_{\pm 0.00}$ \\
	\textit{\quad = 0.5} & 0.75$_{\pm 0.01}$ & 0.17$_{\pm 0.01}$ & 9.01$_{\pm 0.11}$ & 0.21$_{\pm 0.02}$ & 0.02$_{\pm 0.00}$ \\
	Weight decay \\
	\textit{\quad = 0.05} & 0.7